To run via slurm, first output the ipynb notebook to python using `jupyter nbconvert --to script optimization_model.ipynb`

In [14]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import re

## Load precomputed combinations to save computation time

In [13]:
comb_df = pd.read_pickle("../precompute_combinations/precomputed_combinations_df.pkl")

def choose(n, k):
    index = n * 1201 + k
    return comb_df.at[index, "n_choose_k"]

## Probabilities based on our data analysis

In [188]:
# Probability of a goal in a given 10 second period during 5v5
p_a1 = 5.18 * 10**-3
p_b1 = p_a1

# Probability of the other team scoring in a given 10 second period during 6v5 increases
p_b2 = 7.85 * 10**-3

# Probability of our team scoring during 6v5 also increases, though not as much
# p_a2 = 7.22 * 10**-3 # Calculated from our data. Results in "Always pull the goalie" strategy
# p_a2 = 6 * 10**-3 # This is a bit more conservative and allowed more interesting results

# Try varying our probability of scoring during 6v5
# p_a2_options = [4.75 * 10**-3, 5.1 * 10**-3, 5.35 * 10**-3, 5.5 * 10**-3, 6 * 10**-3]
p_a2 = 5.35 * 10**-3 # This gave something interesting with 2 minutes left in the game

## Inputs to model

In [190]:
# Let's say the current score is 1 - 2
a_0 = 1
# b_0 = 2

their_score_options = [1, 2, 3, 4]

# 2 minutes left
minutes_left = 2



## Brute force every choice of $s$

Each team is allowed to score either 0 or 1 goals in each 10 second period.

Launching via slurm on SSCC clusters to speed up

In [191]:
decisions_df = pd.DataFrame(columns = [
        'pull_time',
        'prob_success',
        'compute_time'])

results = []

for b_0 in their_score_options:
    T = minutes_left * 6  # number of 10 second periods left in the game

    #  Try out each possible time to pull the goalie
    for s in range(0, T + 1):

        # Timing for performance metrics
        start_time = time.time()

        prob_success = 0

        # Tracking num_arrangements for complexity considerations
        num_arrangements = 0

        # Number of goals scored by other team before pulling the goalie
        for g_b1 in range(0, s+1):
            prob_gb1 = choose(s, g_b1) * (p_b1 ** g_b1) * ((1 - p_b1) ** (s - g_b1))

            # Number of goals scored by our team before pulling the goalie
            for g_a1 in range(0, s+1):
                prob_ga1 = choose(s, g_a1) * (p_a1 ** g_a1) * ((1 - p_a1) ** (s - g_a1))

                # Number of goals scored by other team after pulling the goalie
                for g_b2 in range(0, T - s+1):
                    prob_gb2 = choose(T - s, g_b2) * (p_b2 ** g_b2) * ((1 - p_b2) ** (T - s - g_b2))

                    # Number of goals scored by our team after pulling the goalie
                    for g_a2 in range(0, T - s+1):
                        num_arrangements += 1
                        prob_ga2 = choose(T - s, g_a2) * (p_a2 ** g_a2) * ((1 - p_a2) ** (T - s - g_a2))

                        # Probability of this event
                        prob = prob_gb1 * prob_ga1 * prob_gb2 * prob_ga2

                        # Update the score
                        a = a_0 + g_a1 + g_a2
                        b = b_0 + g_b1 + g_b2

                        # Success if we win or tie
                        success = (a >= b)

                        prob_success = prob_success + prob * success
                        # display(f"    prob_ga1={prob_ga1}, prob_gb1={prob_gb1}, prob_ga2={prob_ga2}, prob_gb2={prob_gb2}")
                        # display(
                        #     f"  ga1={g_a1}, gb1={g_b1}, ga2={g_a2}, gb2={g_b2}, success: {success}, probability: {prob}"
                        # )

        prob_success = round(prob_success * 100, 4)

        end_time = time.time()
        elapsed_time = end_time - start_time
        elapsed_time = round(elapsed_time, 2)

        # display(f"Pull time: {s * 10} seconds, num_arrangements: {num_arrangements} Probability of success: {prob_success}, Compute time: {elapsed_time}")

        results.append({
            "b_0": b_0,
            "pull_time": s*10,
            "prob_success": prob_success,
            "compute_time": elapsed_time
        })

decisions_df = pd.DataFrame(results)
display(decisions_df)

decisions_df.to_pickle("results/2minleft_vary_score.pkl")

,b_0,pull_time,prob_success,compute_time
0,1,0,91.5155,0.00
1,1,10,91.7450,0.00
2,1,20,91.9751,0.00
3,1,30,92.2059,0.01
4,1,40,92.4374,0.01
5,1,50,92.6695,0.01
6,1,60,92.9023,0.01
7,1,70,93.1358,0.01
8,1,80,93.3699,0.01
9,1,90,93.6047,0.00


## Select the optimal choice

In [161]:
optimal_index = decisions_df["prob_success"].idxmax()
optimal_time = decisions_df.loc[optimal_index, "pull_time"]
optimal_prob = decisions_df.loc[optimal_index, "prob_success"]

print(f"Optimal time to pull the goalie: {optimal_time} seconds, resulting in a probability of {optimal_prob} of winning or tying the game.")

Optimal time to pull the goalie: 30 seconds, resulting in a probability of 5.688 of winning or tying the game.


## Function to read partial output from slurm out txt file

In [ ]:
# Parse partial results from slurm out file
results = []
pattern = r"Pull time: (\d+), num_arrangements: (\d+) Probability of success: ([\d.]+), Compute time: ([\d.]+)"

# Read in lines from temp.txt
with open("results/temp.txt") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()

        if line.startswith("Pull time:"):
            match = re.search(pattern, line)
            pull_time = int(match.group(1))
            num_arrangements = int(match.group(2))
            prob_success = float(match.group(3))
            compute_time = float(match.group(4))
            results.append({
                "pull_time": pull_time,
                "num_arrangements": num_arrangements,
                "prob_success": prob_success,
                "compute_time": compute_time
            })

# Create a DataFrame from the results
decisions_df = pd.DataFrame(results)
display(decisions_df)
decisions_df.to_pickle("results/temp.pkl")